## Reconstructing a Mimulus naiandinus genome from hybrid genomes

<a id="contents"></a>
### Table o' contents

[Align our reads to the reference genome](#align)

<a id="read_quality"></a>
### Look at read qualities

Let's use fastx tools to take a look at the average read qualities.

In [6]:
## path to our fastq files
fpath='/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/'

Fastx requires that we calculate the statistics first:

In [5]:
for i in $fpath*"fastq"
do
echo $i
$ftd"fastx_quality_stats" -Q33 -i $i -o $i"-Stats.txt"
echo $i done!
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq done!
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002

Then graph:

In [8]:
for i in $fpath*Stats.txt; do
fastq_quality_boxplot_graph.sh -i $i -o $i"quality.eps" -p
echo $i"quality.eps"
done


/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S1_S16_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S2_S17_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S3_S18_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S4_S19_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S5_S20_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/JP-S6_S21_L002_R1_001.fastq-Stats.txtquality.eps
/Users/danthomas/Documents/naiandinus_gen/for_dan/Cooley_3315_160429B1/merged.fastq-Stats.txtquality.eps


What do these look like? Here is our merged file, that has all of the reads:

![read_quality](for_dan/Cooley_3315_160429B1/merged.fastq-Stats.txtquality.png)

Looks great, high quality reads, with almost no variance. 

<a id="align"></a>
### Align our reads to the reference genome

We'll use [bowtie2](http://bowtie-bio.sourceforge.net/bowtie2/index.shtml) to align our reads to our reference genome. The reference genome needs to be indexed in the way that bowtie2 wants, first:

In [10]:
bowtie2-build -f /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta Mlu

Settings:
  Output files: "Mlu.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  /Users/danthomas/Documents/naiandinus_gen/Luteus_genome/Mimulus_luteus.fasta
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:09
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:04
bmax according to bmaxDivN setting: 98519930
Using parameters --bmax 73889948 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these paramet

Then we can loop through our fastq files and do alignments against this reference genome. This results in a sequence alignment map (SAM).

In [11]:
cd /Users/danthomas/Documents/naiandinus_gen/Luteus_genome

for i in $fpath*001.fastq; do
    j=$(basename $i)
    k=${j/fastq/sam}
    bowtie2 --local  -x Mlu -U $i -S /Users/danthomas/Documents/naiandinus_gen/SAM/$k \
done

These SAM files look like this:

In [ ]:
head 